In [2]:
import numpy as np
import pandas as pd

In [3]:
market_data = pd.read_excel("../../raw_data/market_data.xlsx")
market_data.drop(index=market_data.index[0], axis=0, inplace=True)
market_data = market_data.reset_index(drop = True)

In [4]:
market_data.head()

,Time (UTC+10),Regions NSW Trading Price ($/MWh),Regions SA Trading Price ($/MWh),Regions TAS Trading Price ($/MWh),Regions VIC Trading Price ($/MWh),Regions NSW Trading Total Intermittent Generation (MW),Regions SA Trading Total Intermittent Generation (MW),Regions TAS Trading Total Intermittent Generation (MW),Regions VIC Trading Total Intermittent Generation (MW),Regions NSW Operational Demand (MW),Regions SA Operational Demand (MW),Regions TAS Operational Demand (MW),Regions VIC Operational Demand (MW)
0,2018-01-01 00:30:00,91.86,107.17,92.28,92.46,0.15,43.07,118.73,131.68,6974,1359,1082,4398
1,2018-01-01 01:00:00,88.83,103.31,87.53,87.62,0.13,41.67,110.48,119.98,6790,1316,1071,4238
2,2018-01-01 01:30:00,73.62,88.20,76.29,73.08,0.14,42.15,120.09,123.86,6536,1240,1067,4112
3,2018-01-01 02:00:00,71.49,85.24,75.10,70.18,0.16,38.31,114.64,132.72,6339,1194,1061,3956
4,2018-01-01 02:30:00,69.27,81.75,72.92,67.43,0.16,33.39,112.90,120.73,6195,1163,1060,3833


### Adding Period

In [5]:
# Create a period for a whole day which are 48 as
# Spot prices are taken by the 30 minutes mark.
period = []
count = 1
for i in range(1, len(market_data) + 1):
    period.append(count)
    count += 1
    if (i % 48) == 0:
        count = 1
        
market_data['Period'] = pd.Series(period)
market_data['Time (UTC+10)'] = pd.to_datetime(market_data['Time (UTC+10)'])

In [8]:
# TRAINING SET
# Training period IS FROM 01/01/2018 to 30/06/2021 PROVIDED IN THE SPEC (3.5 YEARS)
train_start_period = '2018-01-01 00:30:00'
train_end_period   = '2021-07-01 00:00:00'

train_period = (market_data['Time (UTC+10)'] >= train_start_period) & \
               (market_data['Time (UTC+10)'] <= train_end_period)

train_data = market_data[train_period]

In [12]:
train_data

,Time (UTC+10),Regions NSW Trading Price ($/MWh),Regions SA Trading Price ($/MWh),Regions TAS Trading Price ($/MWh),Regions VIC Trading Price ($/MWh),Regions NSW Trading Total Intermittent Generation (MW),Regions SA Trading Total Intermittent Generation (MW),Regions TAS Trading Total Intermittent Generation (MW),Regions VIC Trading Total Intermittent Generation (MW),Regions NSW Operational Demand (MW),Regions SA Operational Demand (MW),Regions TAS Operational Demand (MW),Regions VIC Operational Demand (MW),Period
0,2018-01-01 00:30:00,91.86,107.17,92.28,92.46,0.15,43.07,118.73,131.68,6974,1359,1082,4398,1
1,2018-01-01 01:00:00,88.83,103.31,87.53,87.62,0.13,41.67,110.48,119.98,6790,1316,1071,4238,2
2,2018-01-01 01:30:00,73.62,88.20,76.29,73.08,0.14,42.15,120.09,123.86,6536,1240,1067,4112,3
3,2018-01-01 02:00:00,71.49,85.24,75.10,70.18,0.16,38.31,114.64,132.72,6339,1194,1061,3956,4
4,2018-01-01 02:30:00,69.27,81.75,72.92,67.43,0.16,33.39,112.90,120.73,6195,1163,1060,3833,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61291,2021-06-30 22:00:00,95.21,112.19,117.54,92.52,29.46,31.90,109.11,216.45,9214,1850,1332,5874,44
61292,2021-06-30 22:30:00,71.81,95.28,63.15,67.46,27.71,20.82,103.20,224.00,9022,1766,1289,5653,45
61293,2021-06-30 23:00:00,75.17,87.72,60.16,60.16,12.08,11.21,79.69,211.81,8804,1672,1269,5397,46
61294,2021-06-30 23:30:00,80.41,88.37,71.57,73.85,3.41,6.33,81.11,190.95,8654,1587,1231,5487,47


In [11]:
# TESTING SET
# TESTING PERIOD IS FROM 01/07/2021 TO 14/08/2021 PROVIDED IN THE SPEC (6 WEEKS)
test_start_period = '2021-07-01 00:30:00'
test_end_period = '2021-08-15 00:00:00'

test_period = (market_data['Time (UTC+10)'] >= test_start_period) & \
               (market_data['Time (UTC+10)'] <= test_end_period)

test_data = market_data[test_period]

In [13]:
test_data

,Time (UTC+10),Regions NSW Trading Price ($/MWh),Regions SA Trading Price ($/MWh),Regions TAS Trading Price ($/MWh),Regions VIC Trading Price ($/MWh),Regions NSW Trading Total Intermittent Generation (MW),Regions SA Trading Total Intermittent Generation (MW),Regions TAS Trading Total Intermittent Generation (MW),Regions VIC Trading Total Intermittent Generation (MW),Regions NSW Operational Demand (MW),Regions SA Operational Demand (MW),Regions TAS Operational Demand (MW),Regions VIC Operational Demand (MW),Period
61296,2021-07-01 00:30:00,190.22,96.47,99.43,90.51,5.26,4.92,77.71,175.53,8402,1679,1154,5192,1
61297,2021-07-01 01:00:00,79.17,87.88,70.95,73.91,5.05,4.73,73.68,130.56,8256,1638,1161,5016,2
61298,2021-07-01 01:30:00,68.24,102.82,70.46,33.79,4.62,7.19,76.40,120.55,8031,1566,1146,4837,3
61299,2021-07-01 02:00:00,72.64,106.60,55.53,43.57,4.12,7.45,78.61,125.84,7765,1502,1117,4661,4
61300,2021-07-01 02:30:00,64.84,88.53,55.97,45.90,3.30,4.63,80.06,108.33,7476,1457,1129,4523,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63451,2021-08-14 22:00:00,50.84,59.16,7.54,49.93,8.74,36.42,154.69,182.52,8491,1718,1255,5492,44
63452,2021-08-14 22:30:00,66.85,80.01,10.52,62.86,7.52,51.83,156.09,206.23,8376,1665,1244,5344,45
63453,2021-08-14 23:00:00,55.64,77.76,7.63,32.26,7.69,42.04,160.30,215.20,8194,1614,1207,5204,46
63454,2021-08-14 23:30:00,52.25,76.47,7.52,25.10,8.35,38.04,167.00,226.95,8022,1573,1163,5268,47


In [14]:
train_data.to_excel('../../preprocessed_data/Second Algorithm/training_set.xlsx', index = False)
test_data.to_excel('../../preprocessed_data/Second Algorithm/testing_set.xlsx', index = False)